# Characters Embedding

In [1]:

from numpy import array
from pickle import load, dump
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import numpy as np

import random
import os
import sys
import re


In [2]:
# Metaparameters
PATH_DATASET = "./dataset/"
PATH_MODEL = "./models/"
PATH_CATEGORIES = PATH_DATASET + "categories/"
SEQUENCE_LENGTH = 20  # The length of the sentences into which we will divide the text.
LSTM_CELLS = 500  # The number of neurons in the LSTM. If you have to learn a very long text, it is better to increase this number, if the text to be learned is small, much less may be enough (even less than 100).
filename = 'all_categories'  #

In [ ]:
def noise_removal_old(text):
    text = re.sub('@', '', text)    
    text = re.sub('#', '', text)    
    text = re.sub('°', '', text)
    text = re.sub('§', '', text)
    text = re.sub('ɑ', '', text)
    text = re.sub('ɔ', '', text)
    text = re.sub('ə', '', text)
    text = re.sub('ʃ', '', text)
    text = re.sub('ά', '', text)
    text = re.sub('έ', '', text)
    text = re.sub('ή', '', text)
    text = re.sub('α', '', text)
    text = re.sub('β', '', text)
    text = re.sub('γ', '', text)
    text = re.sub('δ', '', text)
    text = re.sub('ε', '', text)
    text = re.sub('η', '', text)
    text = re.sub('θ', '', text)
    text = re.sub('ι', '', text)
    text = re.sub('κ', '', text)
    text = re.sub('λ', '', text)
    text = re.sub('μ', '', text)
    text = re.sub('ν', '', text)
    text = re.sub('ξ', '', text)
    text = re.sub('ο', '', text)
    text = re.sub('π', '', text)
    text = re.sub('ρ', '', text)
    text = re.sub('ς', '', text)
    text = re.sub('σ', '', text)
    text = re.sub('τ', '', text)
    text = re.sub('υ', '', text)
    text = re.sub('φ', '', text)
    text = re.sub('χ', '', text)
    text = re.sub('ψ', '', text)
    text = re.sub('ω', '', text)
    #text = re.sub('<[/\'\"A-Za-z0-9=A-Za-z0-9\'\"]*>', '', text) # rimozione tag
    text = re.sub('<[^>]*>', '', text) # rimozione tag   
    #text = re.sub('\...', '', text)
    return text

In [3]:
def noise_removal(text):
    text = re.sub('<[^>]*>', '', text) # rimozione tag
    text = re.sub('[^a-zA-Z0-9 .:,;èòóàùúéìí]*', '', text)     
    return text

In [4]:
for c in range(33, 256):
    print(c, chr(c))

33 !
34 "
35 #
36 $
37 %
38 &
39 '
40 (
41 )
42 *
43 +
44 ,
45 -
46 .
47 /
48 0
49 1
50 2
51 3
52 4
53 5
54 6
55 7
56 8
57 9
58 :
59 ;
60 <
61 =
62 >
63 ?
64 @
65 A
66 B
67 C
68 D
69 E
70 F
71 G
72 H
73 I
74 J
75 K
76 L
77 M
78 N
79 O
80 P
81 Q
82 R
83 S
84 T
85 U
86 V
87 W
88 X
89 Y
90 Z
91 [
92 \
93 ]
94 ^
95 _
96 `
97 a
98 b
99 c
100 d
101 e
102 f
103 g
104 h
105 i
106 j
107 k
108 l
109 m
110 n
111 o
112 p
113 q
114 r
115 s
116 t
117 u
118 v
119 w
120 x
121 y
122 z
123 {
124 |
125 }
126 ~
127 
128 
129 
130 
131 
132 
133 
134 
135 
136 
137 
138 
139 
140 
141 
142 
143 
144 
145 
146 
147 
148 
149 
150 
151 
152 
153 
154 
155 
156 
157 
158 
159 
160  
161 ¡
162 ¢
163 £
164 ¤
165 ¥
166 ¦
167 §
168 ¨
169 ©
170 ª
171 «
172 ¬
173 ­
174 ®
175 ¯
176 °
177 ±
178 ²
179 ³
180 ´
181 µ
182 ¶
183 ·
184 ¸
185 ¹
186 º
187 »
188 ¼
189 ½
190 ¾
191 ¿
192 À
193 Á
194 Â
195 Ã
196 Ä
197 Å
198 Æ
199 Ç
200 È
201 É
202 Ê
203 Ë
204 Ì
205 Í
206 Î
207 Ï
208 Ð
209 Ñ
210 Ò

In [5]:
testo = " <div><h1>titolo </h1> <div id='app'>§§ °paragrafo° §§ <p> Imo this is ok. b4n</div></div>"
print(noise_removal(testo))

 titolo   paragrafo   Imo this is ok. b4n


In [6]:
def space_removal(text):
    text = text.replace("    ", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.replace("  ", " ")    
    return text

In [7]:
def normalization(text):
    text = text.replace(":-)", "smile")
    text = text.replace(":)", "smile")
    text = text.replace(":D", "smile")
    text = text.replace("b4n", "bye for now")    
    text = text.replace("b4", "before")
    text = text.replace("lmk", "let me know")
    text = text.replace("nmd", "nevermind")
    text = text.replace("imo", "in my opinion")
    text = text.replace("tia", "thanks in advance")
    text = text.replace("?4u", "i have a question for you")
    text = text.replace("\n", " ")
    text = text.replace(",", " , ")
    text = text.replace(".", " . ")
    text = text.replace(":", " : ")
    text = text.replace(";", " ; ")
    text = space_removal(text)
    return text

In [8]:
def preprocess_text(text):
    text = text.lower()
    text = normalization(text)
    text = noise_removal(text)
    text = space_removal(text)
   # text = text.split(' ')  # non vanno divise
   # text = stopword_removal(text) # meglio non rimuoverle
    
    return text

In [9]:
print(preprocess_text(testo))

 titolo paragrafo in my opinion this is ok . bye for now


In [10]:
print(os.listdir(PATH_DATASET))

['dataset_cat.csv', 'economics', 'literature', 'philosophy', 'physics', 'politics', 'psychology', 'sport']


In [11]:
categories = []
with os.scandir(PATH_DATASET) as it:
    for entry in it:
        if not entry.name.startswith('.') and entry.is_dir():
            categories.append(entry.name)
print("categories:", categories)


categories: ['economics', 'literature', 'philosophy', 'physics', 'politics', 'psychology', 'sport']


In [12]:
categories = ['literature']

In [13]:
def import_dataset(name_file, categories):
    count = 0
    with open(PATH_DATASET + name_file + ".txt", 'w', newline='', encoding='UTF-8') as file_out:
        for idx, cat in enumerate(categories):
            print("\n *** Importing", cat)
            path_cat = PATH_DATASET + cat + "/"
            files = os.listdir(path_cat)
            print(files)
            for f in files:
                with open(path_cat + f, 'r', encoding='UTF-8') as file_in:
                    file_text = file_in.read()
                    file_out.write(file_text)
            print(count)
 

In [14]:

import_dataset("all_categories", ['literature'])


 *** Importing literature
['History of literature.txt', 'Literature review.txt', 'Literature.txt', 'Nobel Prize in Literature.txt', 'Victorian literature.txt', 'Western literature.txt', 'World literature.txt']
0


In [15]:
def get_and_preprocess_file_text(file_name):
    """
    Load the file's text and preprocess it
    :param file_name: the file to open
    :return: nothing
    """
    with open(PATH_DATASET + file_name, 'r', encoding='UTF-8') as file: # open the file as read only
        text = file.read()  # read the file and get text
        text = preprocess_text(text)  # preprocess text (this step can also be skipped)

    return text  # return the text of file

In [16]:
def split_in_sequences(text):
    """
    Split text into sequences of characters SEQUENCE_LENGTH characters long
    the sequences are created by shifting, for each subsequent sequence, by one character with respect to the previous one
    :param text: the text to split
    :return: the list of sequences
    """
    length = SEQUENCE_LENGTH  # the length of the sequences set by the metaparameter
    sequences = []  # The list that will contain the sequences
    for i in range(len(text) - length):  # repeats the following operations with i taking values from 0 to the length of the text passed - SEQUENCE_LENGTH
        seq = text[i: length + i + 1]  # The first sequence will consist of the first characters length of the text +1, then the same length starting from the second and so on ...
        sequences.append(seq)  # Add the sequence to the list
    print('Number of sequences: ' + str(len(sequences)))  # print the number of sequences
    return sequences  # the list of sequences


In [ ]:
def get_saved_model(id=''):
    try: 
        model = load_model(PATH_MODEL + 'model'+id+'.h5')  
        mapping = load(open(PATH_MODEL + 'mapping'+id+'.pkl', 'rb')) 
        print("Continuing train on saved model... ")  
    except Exception as e: 
        print("Files non found ", e) 
    return

In [17]:
def get_model(num_char):
    """
    Prepare the model based on the number of characters of the mapping
    (which determines the output layer and the second dimension of the input)
    and the length of the sequences.
    :param num_char: the number of characters of the mapping
    :return: the model ready for training
    """
    model = Sequential()  # Let's create an instance of the Keras object that allows you to create a network in a very simple way
    model.add(LSTM(LSTM_CELLS, input_shape=(SEQUENCE_LENGTH, num_char)))  # The number of neurons is the LSTM_CELLS metaparameter defined at the beginning of the script, the size of the input is given by the number of characters in the mapping and the length of the sequences
    model.add(Dense(num_char, activation='softmax'))  # The size of the output is determined by the number of characters in the mapping
    print(model.summary())  # print the model schema
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # we compile the model
    return model


In [21]:
# The file containing the text on which the training will be carried out can be as short or long as you like.
# Adjust the metaparameters also according to this.
# If very short LSTM_CELLS can be even 100 or less, if it is about 50,000 characters it is recommended to set it to at least 300, if about 100,000 LSTM_CELLS it should be at least 500 and so on...


text = get_and_preprocess_file_text(filename + ".txt")  # Load the file and do a minimal preprocessing.
sequences = split_in_sequences(text)  # Split text into sequences of characters


Number of sequences: 138631


In [22]:
# To get an ordered list of the set of characters present in the text
chars = sorted(list(set(text)))  # First we get a set from the text (so each different character is taken only once), then this set of characters is transformed into a list and sorted
mapping = dict((c, i) for i, c in enumerate(chars))  # Let's create a dictionary that maps characters
print(mapping)



{' ': 0, ',': 1, '.': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, ':': 13, ';': 14, 'a': 15, 'b': 16, 'c': 17, 'd': 18, 'e': 19, 'f': 20, 'g': 21, 'h': 22, 'i': 23, 'j': 24, 'k': 25, 'l': 26, 'm': 27, 'n': 28, 'o': 29, 'p': 30, 'q': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'w': 37, 'x': 38, 'y': 39, 'z': 40, 'è': 41, 'é': 42, 'í': 43, 'ó': 44}


In [23]:
model = get_model(len(mapping))  # Prepare the model based on the number of characters of the mapping and the length of the sequences.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 500)               1092000   
                                                                 
 dense (Dense)               (None, 45)                22545     
                                                                 
Total params: 1,114,545
Trainable params: 1,114,545
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
encoded_sequences = []  # a new list that will contain the character sequences encoded according to the mapping
for line in sequences:  # We go through all the sequences contained in the list and prepared previously
    encoded_seq = [mapping[char] for char in line]  # Each character in the sequence is replaced by the corresponding integer based on the mapping
    encoded_sequences.append(encoded_seq)  # The new sequence (now no longer of characters but of numbers) is added to the list



In [25]:
encoded_sequences = array(encoded_sequences)  # We transform the list of lists into a numpy matrix.
X, y = encoded_sequences[:, :-1], encoded_sequences[:, -1]  # We derive a matrix and a vector. The matrix X is made from the same rows of encoded_sequences without the last element, the vector only from the last element of those rows.



In [26]:
encoded_sequences = [to_categorical(x, num_classes=len(mapping)) for x in X]  # Convert encoding numbers of sequences to one-hot vectors
X = array(encoded_sequences)
y = to_categorical(y, num_classes=len(mapping))  # Convert encoding numbers of target characters to one-hot vectors
print("X.shape: " + str(X.shape))
print("y.shape: " + str(y.shape))
print("SEQUENCE_LENGTH: " + str(SEQUENCE_LENGTH), "len(mapping): " + str(len(mapping)))


X.shape: (138631, 20, 45)
y.shape: (138631, 45)
SEQUENCE_LENGTH: 20 len(mapping): 45


In [27]:
epochs_def = 15
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=3, min_delta=0.00001)  # By setting the EarlyStopping we ask that the train be stopped when the set conditions occur (in our case there is no appreciable improvement for 4 epochs)
callback = [es]  # To set the early stopping we have to put it in a list (which could also contain other objects for other settings, here https://keras.io/api/callbacks/ more details are available)

with tf.device("/cpu:0"):
    model.fit(X, y, epochs=epochs_def, verbose=1, batch_size=64, callbacks=callback)  # With this call we start training the network. We pass them our examples (X) and the desired answers (y), the number of epochs and the callbacks we want to set (in our case only early stopping).

model.save(PATH_MODEL + 'model.h5')  # save the model to file
dump(mapping, open(PATH_MODEL + 'mapping.pkl', 'wb'))  # And save the mapping with which the text used for training was encoded (and it will be used for text to be used for predict)


Epoch 1/15
2167/2167 [==============================] - 427s 196ms/step - loss: 2.2147 - accuracy: 0.3584
Epoch 2/15
2167/2167 [==============================] - 425s 196ms/step - loss: 1.7126 - accuracy: 0.4971
Epoch 3/15
2167/2167 [==============================] - 427s 197ms/step - loss: 1.4893 - accuracy: 0.5621
Epoch 4/15
2167/2167 [==============================] - 427s 197ms/step - loss: 1.3403 - accuracy: 0.6036
Epoch 5/15
2167/2167 [==============================] - 429s 198ms/step - loss: 1.2120 - accuracy: 0.6374
Epoch 6/15
2167/2167 [==============================] - 435s 201ms/step - loss: 1.0902 - accuracy: 0.6724
Epoch 7/15
2167/2167 [==============================] - 439s 203ms/step - loss: 0.9759 - accuracy: 0.7029
Epoch 8/15
2167/2167 [==============================] - 436s 201ms/step - loss: 0.8658 - accuracy: 0.7359
Epoch 9/15
2167/2167 [==============================] - 440s 203ms/step - loss: 0.7639 - accuracy: 0.7665
Epoch 10/15
2167/2167 [=======================

In [28]:
epochs_def = 15
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=3, min_delta=0.001)  # By setting the EarlyStopping we ask that the train be stopped when the set conditions occur (in our case there is no appreciable improvement for 4 epochs)
callback = [es]  # To set the early stopping we have to put it in a list (which could also contain other objects for other settings, here https://keras.io/api/callbacks/ more details are available)

with tf.device("/cpu:0"):
    model.fit(X, y, epochs=epochs_def, verbose=1, batch_size=128, callbacks=callback)  # With this call we start training the network. We pass them our examples (X) and the desired answers (y), the number of epochs and the callbacks we want to set (in our case only early stopping).

model.save(PATH_MODEL + 'model.h5')  # save the model to file
dump(mapping, open(PATH_MODEL + 'mapping.pkl', 'wb'))  # And save the mapping with which the text used for training was encoded (and it will be used for text to be used for predict)


Epoch 1/15
1084/1084 [==============================] - 308s 284ms/step - loss: 0.2414 - accuracy: 0.9354
Epoch 2/15
1084/1084 [==============================] - 305s 282ms/step - loss: 0.1591 - accuracy: 0.9631
Epoch 3/15
1084/1084 [==============================] - 306s 282ms/step - loss: 0.1761 - accuracy: 0.9553
Epoch 4/15
1084/1084 [==============================] - 306s 282ms/step - loss: 0.1994 - accuracy: 0.9443
Epoch 5/15
1084/1084 [==============================] - 305s 281ms/step - loss: 0.1647 - accuracy: 0.9563
Epoch 5: early stopping


In [29]:
epochs_def = 4
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=3, min_delta=0.01)  # By setting the EarlyStopping we ask that the train be stopped when the set conditions occur (in our case there is no appreciable improvement for 4 epochs)
callback = [es]  # To set the early stopping we have to put it in a list (which could also contain other objects for other settings, here https://keras.io/api/callbacks/ more details are available)

with tf.device("/cpu:0"):
    model.fit(X, y, epochs=epochs_def, verbose=1, batch_size=256, callbacks=callback)  # With this call we start training the network. We pass them our examples (X) and the desired answers (y), the number of epochs and the callbacks we want to set (in our case only early stopping).

model.save(PATH_MODEL + 'model.h5')  # save the model to file
dump(mapping, open(PATH_MODEL + 'mapping.pkl', 'wb'))  # And save the mapping with which the text used for training was encoded (and it will be used for text to be used for predict)


Epoch 1/4
542/542 [==============================] - 265s 489ms/step - loss: 0.0970 - accuracy: 0.9790
Epoch 2/4
542/542 [==============================] - 265s 489ms/step - loss: 0.0490 - accuracy: 0.9907
Epoch 3/4
542/542 [==============================] - 264s 487ms/step - loss: 0.0417 - accuracy: 0.9913
Epoch 4/4
542/542 [==============================] - 264s 488ms/step - loss: 0.0412 - accuracy: 0.9915


In [18]:
def generate_seq(model, mapping, seed_text, n_chars, one_word=False):
    """
    It uses the passed model and mapping to generate the indicated number of characters
    :param model:       The model on which the predict method will be called
    :param mapping:     The mapping to use for the sequence encoding
    :param seed_text:   The test that will be completed
    :param n_chars:     The desired length of the final sentence
    :param one_word:    If True only complete the word and ignore seq_length (default: False)
    :return:    The generated sequence
    """

    seed_text = seed_text.lower()
    return_text = seed_text
    for _ in range(n_chars+SEQUENCE_LENGTH):  # The following block will be executed for the number of times indicated by n_chars, then until a sentence of this length (plus that of the seed_text) is formed
        encoded = [mapping[char] for char in seed_text] # encode the characters as integers based on param mapping
        encoded = pad_sequences([encoded], maxlen=SEQUENCE_LENGTH, truncating='pre')  # truncate sequences to length of SEQUENCE_LENGTH
        encoded = to_categorical(encoded, num_classes=len(mapping))  # Convert encoding numbers to one-hot vectors
        pred_char = model.predict(encoded, verbose=0)  # The predict on the model is called to obtain the predicted character
        #print(pred_char)
        #print(tf.nn.softmax(pred_char))
        #print(max(pred_char[0]))
        #print(np.argmax(pred_char[0]))
        out_char = ''
        for char, index in mapping.items():  # The character corresponding to the one produced by the predict is searched in the mapping        
            if index == np.argmax(pred_char[0]):
                out_char = char
                #print("index", index, char)
                break
        seed_text += out_char  # The character thus obtained is added to the sequence that is being generated
        return_text += out_char
        if len(seed_text)>SEQUENCE_LENGTH:
            seed_text = seed_text[len(seed_text)-SEQUENCE_LENGTH:]
        if (one_word and out_char == ' ') or (len(return_text)>n_chars and out_char == ' '):  # If the one_word parameter was set to True and the predicted character is a space, the word has been completed
            break  # And it stops

        #print(seed_text) 
    return return_text  # The generated sequence


In [19]:
def complete_text(text, model, mapping, one_word=False, length=SEQUENCE_LENGTH*2):
    """
    A function that completes the passed text using the passed template and mapping
    :param text: The test that will be completed
    :param model: The model on which the predict will be called
    :param mapping: The mapping to use for the sequence encoding
    :param one_word: If set to True it will only complete the last word
    :param length: The length of the final sentence (unless one_word = True parameter in which case the sentence could be shorter)
    :return:
    """
    text = preprocess_text(text)  # a minimal preprocessing.
    if len(text) > SEQUENCE_LENGTH:  # If the proposed sentence is longer than SEQUENCE_LENGTH
        text = text[(len(text)-SEQUENCE_LENGTH):len(text)]  # Part of the length of SEQUENCE_LENGTH will be taken
    #if len(text) < SEQUENCE_LENGTH:  # If the proposed sentence is shorter than SEQUENCE_LENGTH
    #    text = '             '[:(SEQUENCE_LENGTH - len(text))] + text  # spaces are added until the right length is reached
    #    print('|' + text + '|')  # Print of the modified sentence
    seq = generate_seq(model, mapping, text, length, one_word)  # The function that actually completed the sentence is called
    print(seq)  # Print of the completed sentence



In [30]:
# Test
complete_text('hello worl', model, mapping, True)

hello world 


In [31]:
complete_text('hello wo', model, mapping, True)

hello works 


In [32]:
complete_text('the w', model, mapping, True)

the works 


In [33]:
complete_text('The work', model, mapping)

the works of the will of swedish industrialist 


In [34]:
complete_text('Whose work', model, mapping)

whose works were published in the united 


In [35]:
complete_text('this work', model, mapping)

this works the first fact ard industry , 


In [36]:
complete_text('litera', model, mapping)

literah part of the religious writer john 


In [37]:
complete_text('literatu', model, mapping)

literaturan from soun 1011109 , ed . the 


In [38]:
complete_text('One such view is', model, mapping)

one such view is a process that continues 


In [39]:
complete_text('Accredited assoc', model, mapping)

accredited associated with academicoriented 


In [20]:
def test_complete_text(one_word=False, model=None, mapping=None):
    """
    A test of the model consisting in ask for write some words with the last incomplete (or only the latter) verifying how the model completes it.
    :param one_word: If set to True, the model will just complete the incomplete word, with False it will produce a 3x long sentence SEQUENCE_LENGTH.
    :return: Nothing
    """
    if model==None:
        print("loading saved model...")
        model = load_model(PATH_MODEL + 'model.h5')  # Load model previously saved
    if mapping==None:
        print("loading saved mapping...")
        mapping = load(open(PATH_MODEL + 'mapping.pkl', 'rb'))  # And the mapping he was trained with

    text = 'write a text'
    while len(text) > 1:  # Keep repeating the following instructions until you reply with a return without writing anything
        complete_text(text, model, mapping, one_word)  # The function that completes the passed sentence is called
        text = input("Write a text, please...   \n ")  # It invites you to write the sentence to complete



In [ ]:
test_complete_text(model=model, mapping=mapping)

In [40]:
complete_text('hello ', model, mapping, length=200)

hello . prominent atheist figures such as the late christopher hitchens and richard dawkins have praised the king james version of the bible has been called the most influenthanks in advancelly invented drama : his oresteia trilogy of plays is seen as his crowning achievement . other refiners of playwriting were sophocles and euripides . sophocles is credited with skillfully developing irony as a literary technique , most famously in his play oedipus rex . euripedes , conversely , used plays to challenge 


In [42]:
def continue_text(text, model, mapping, one_word=False, length=SEQUENCE_LENGTH*2):
    seq = ''
    tot = 0
    while tot < length:
        text = preprocess_text(text)  # a minimal preprocessing.
        if len(text) > SEQUENCE_LENGTH:  # If the proposed sentence is longer than SEQUENCE_LENGTH
            text = text[(len(text)-SEQUENCE_LENGTH):len(text)]  # Part of the length of SEQUENCE_LENGTH will be taken
        seq = generate_seq(model, mapping, text, SEQUENCE_LENGTH*2, one_word)  # The function that actually completed the sentence is called
        tot += len(seq)
        print(seq[:len(seq) - (SEQUENCE_LENGTH//2)], end='') 
        text = seq[len(seq) - (SEQUENCE_LENGTH//2):]
    print('') 



In [44]:
continue_text('hello ', model, mapping, length=1000)

hello . prominent atheist figures such as the renaissance , the focus on 19 , 000 and 20 , 000 and 2018 , produced the most outstanding that the recest of the academy was not in any condition to credibly jougnes to several earlier works 1850 , which had been states and will have a centain contribution . the products of distant lands and cormedtated to the world betodition . the religious of th
